# Introduction
 In this assignment, you will do exercises on arcpy.Exists(), arcpy.Walk(), arcpy.da.Describe(), python list comprehension, arcpy.da.SearchCursor, and arcpy.da.InsertCursor(). In each section, a code block is provided but not ready to run. You need to add this notebook to ArcGIS Pro, modify each block to make it runable, keep the output message , and write an explanation of the code block. 

 Data preparation: 
 Download the points.csv from the data folder. 

## First, let's use ArcToolbox tools to convert the csv to a feature class (10 pnts)

- In ArcGIS Pro, search for "XYTable to Point"
- In the dialog, put the "point.csv" from this folder as the input table
- Name the output something like "point_fromxy"
- Before, you run the model, in the Run button, click the small triangle and select "Copy Python Command"
- Paste your python code below in the code block
- Run the code block to make sure it works
- Compare the python code and the ArcGIS Pro Geoprocessing pane dialog interface 


In [2]:
# To get the 10 points 
# Paste the python code from the above instruction here
# make sure you start with import arcpy



## Commentignt the example code:(20 pnts)
 - Using InsertCursor to do the same thing but project the geometry to a different spatial reference 

- The above code only imports the csv to a feature class but not converting from a GCS to a project coordinate system
- We will use the following code to do the work and project the data to "NAD 1983 USGS Contiguous USA Albers" system with the WKID = 102039
- Look the code up from here: https://epsg.io/102039

In [16]:
import arcpy
import csv
import os

# Edit the path names for your case
csv_file_path = r"C:\Users\leiwang\Documents\geog4057\Homeworks\points.csv"
geodatabase_path = r"C:\Users\leiwang\Documents\ArcGIS\Projects\GEOG4057\GEOG4057.gdb"
feature_class_name = "point_albers"

# Determine the number of fields and field names from the CSV header
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)
    num_fields = len(header)

# Create SpatialReference objects: 
sr_gcs = arcpy.SpatialReference(4269) 
sr_albsers = arcpy.SpatialReference(102039)
#delete if the feature class exists

fcName = os.path.join(geodatabase_path,feature_class_name)

if arcpy.Exists(fcName):
    arcpy.management.Delete(fcName)


# Create a new point feature class

arcpy.CreateFeatureclass_management(geodatabase_path, feature_class_name, "POINT", spatial_reference=sr_albsers)
# Add fields to the feature class based on the CSV header

# first, read the exisint fields 
describe = arcpy.da.Describe(fcName)
field_names = [field.name for field in describe["fields"]]


for field in header:
    if field not in field_names:
        arcpy.AddField_management(fcName, field, "TEXT")

# Create a cursor for inserting point features with additional fields
cursor_fields = ["SHAPE@"] + header
with arcpy.da.InsertCursor(fcName, cursor_fields) as cursor:
    # Read data from the CSV file and create point features with attributes
    with open(csv_file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        
        # Skip the header row (if present)
        next(csv_reader, None)
        
        for table_value in csv_reader:
            x = float(table_value[0]) # note that the first two fields in the csv must be longitude and latitude
            y = float(table_value[1])
            pnt = arcpy.PointGeometry(arcpy.Point(x,y),sr_gcs)
            pnt_alberse = pnt.projectAs(sr_albsers)

            row = (pnt_alberse,) + tuple(table_value)
            cursor.insertRow(row)

print(f"New point feature class '{feature_class_name}' created in '{geodatabase_path}' with {num_fields} fields.")



New point feature class 'point_albers' created in 'C:\Users\leiwang\Documents\ArcGIS\Projects\GEOG4057\GEOG4057.gdb' with 5 fields.


**Edit this block to answer these three questions**

Q1: Line by line, explain what was done in the code. (10 pnts)

Q2: What do the cod 4269 and 102309 represent? (5 pnts)

Q3: Explain what is SHAPE@ (5 pnts)

## Use arcpy.Exist() (10 pnts)

- This code checks the existence of a specified dataset within an ArcGIS workspace.
- Fill the dataset_name and workspace_path variables with the database and the feature class name (points) from the last block
- Print a message indicating whether the dataset exists or not.

In [ ]:
import arcpy
import os

# Prompt for user input
dataset_name = "points"
workspace_path = "Enter the workspace path: "
fullname = os.path.join(workspace_path,dataset_name)
# Check if the dataset exists
if arcpy.Exists(fullname):
    print(f"The dataset '{dataset_name}' exists in the workspace.")
else:
    print(f"The dataset '{dataset_name}' does not exist in the workspace.")

**Edit this block to answer the question**

Q4: Line by line describe what was done by the code in the block above. (10 pnts) 



## Use arcpy.Walk() (10 pnts)

- This code uses arcpy.Walk() to iterate through all feature datasets in the geodatabase you just used.
- Modify the name of "workspace" to make the code work
- Run the code to list all the feature classes within each dataset.


In [ ]:
import arcpy

# Define the workspace
workspace = "C:/path_to_geodatabase.gdb" # note that you need to put a full path name for this workspace, even running it in ArcGIS Pro. 

# Use arcpy.Walk() to iterate through feature datasets
for dirpath, dirnames, filenames in arcpy.da.Walk(workspace, datatype="FeatureClass"):
    for filename in filenames:
        print(f"Feature Class in {dirpath}: {filename}")

**Edit this block to answer the question**

Q5: line by line, describe what was done by the code in the block above. (10 pnts)


## Use List Comprehension (10 pnts)

- The following block uses list comprehension to generate a list of .shp files in a specified folder.
- Choose the folder name from assignment 4 data/paris subfolder where many shapefiles are located and use it for folder_path
- Run the code to print the list of file names


- In the second block, the code combines the os.walk() function with the list comprehension to  list all shapefiles in a folder including subfolders



In [ ]:
import os

# Specify the folder path
folder_path = "C:/path_to_folder"

# Use list comprehension to generate a list of .shp files
shp_files = [file for file in os.listdir(folder_path) if file.endswith(".shp")]

# Print the list of .shp files
print("Shapefiles in the folder:")
for shp_file in shp_files:
    print(shp_file)

In [ ]:
import os
# Specify the folder path
folder_path = "C:/path_to_folder"

print("Shapefiles in the folder:")
for root, dirs, files in os.walk(folder_path):
    shp_files = [file for file in files if file.endswith(".shp")]
    for shp_file in shp_files:
            print(shp_file)

**Edit this block to answer the question**

Q6: Describe what was done by the code in first blocks above. (5 pnts)

Q7: Describe what was done by the code in second blocks above. (5 pnts)

## Use arcpy.da.SearchCursor (10 pnts)

- This code uses arcpy.da.SearchCursor to extract attribute information from a feature class.
- Enter the name of the "points" feature class and the field(s) from the feature class to extract.
- Run and display the extracted data.


In [ ]:
import arcpy

# Prompt for user input
fc_path = "Enter the feature class path: "
fields_to_extract = ["Enter the field(s) to extract (comma-separated): "]

# Use arcpy.da.SearchCursor to extract data
with arcpy.da.SearchCursor(fc_path, fields_to_extract) as cursor:
    print("Extracted Data:")
    for row in cursor:
        print([row[i] for i in range(len(fields_to_extract))])

**Edit this block to answer the question**

Q8: Line by line, explain what was done in the code. 10 pnts 

## Use addField and field Calculator (20 pnts)

- Run the following code block (with "points" feature class added to the last map before you switched into the notebook interface)

In [ ]:
import arcpy
fc = "points"
newfieldName = "all"
arcpy.AddField_management(fc, newfieldName, "DOUBLE")
expression = "sum(!Field1!,!Field2!,!Field3!)"
codeblock = """
def sum(*fields):
    sum = 0
    for field in fields:
        sum += field
    return sum
"""
arcpy.CalculateField_management(fc, newfieldName, expression, "", codeblock)

**Edit this block to answer the questions**

Q9: line by line, describe what was done by the code in the block above. 10 pnts

Q10: Open the attribute table of "points" and check if the attribute table has a new filed "all" and correct values. Right-click the "all" field and click field calculator. Describe what you see in the field calculator interface. Compare the python code versus the field calculator interface. 10 pnts

## Submit the assignment ontime (10 pnts)

- Late penalty is 10 points from here